In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer, AdamW
from datasets import load_dataset
from tqdm import tqdm
import torch


model_name = "deepset/bert-base-cased-squad2"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
dataset=load_dataset('squad',split='train')


In [ ]:
dataset=dataset[:100]
print(len(dataset['question']))

In [ ]:
print(dataset)
answer_starts=[item['answer_start']for item in dataset['answers']]
print(answer_starts)

In [ ]:

train_contexts = dataset['context']
train_questions = dataset['question']  
train_start_positions = answer_starts 


inputs = tokenizer(train_questions, train_contexts, return_tensors='pt', padding=True, truncation=True)
start_positions = torch.tensor(train_start_positions)


optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
batch_size = 10


for epoch in range(num_epochs):
    for i in tqdm(range(0, len(train_questions), batch_size)):
        optimizer.zero_grad()
        batch_inputs = {key: val[i:i+batch_size] for key, val in inputs.items()}
        outputs = model(**batch_inputs, start_positions=start_positions[i:i+batch_size])




model.save_pretrained('fine_tuned_bert_model')